# notebook setup

In [133]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import IPython.display as ipd
import copy
import utils
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, roc_curve, auc, RocCurveDisplay, ConfusionMatrixDisplay, accuracy_score, classification_report

# data preprocessing

## feature data

In [134]:
features_data = utils.load("fma_metadata/features.csv")

In [135]:
features_data.columns, features_data.shape

(MultiIndex([('chroma_cens', 'kurtosis', '01'),
             ('chroma_cens', 'kurtosis', '02'),
             ('chroma_cens', 'kurtosis', '03'),
             ('chroma_cens', 'kurtosis', '04'),
             ('chroma_cens', 'kurtosis', '05'),
             ('chroma_cens', 'kurtosis', '06'),
             ('chroma_cens', 'kurtosis', '07'),
             ('chroma_cens', 'kurtosis', '08'),
             ('chroma_cens', 'kurtosis', '09'),
             ('chroma_cens', 'kurtosis', '10'),
             ...
             (    'tonnetz',      'std', '04'),
             (    'tonnetz',      'std', '05'),
             (    'tonnetz',      'std', '06'),
             (        'zcr', 'kurtosis', '01'),
             (        'zcr',      'max', '01'),
             (        'zcr',     'mean', '01'),
             (        'zcr',   'median', '01'),
             (        'zcr',      'min', '01'),
             (        'zcr',     'skew', '01'),
             (        'zcr',      'std', '01')],
            names=['fe

In [136]:
columns = ['mfcc', 'chroma_cens', 'tonnetz', 'spectral_contrast']
columns.append(['spectral_centroid', 'spectral_bandwidth', 'spectral_rolloff'])
columns.append(['rmse', 'zcr'])
for column in columns:
    ipd.display(features_data[column].head().style.format('{:.2f}'))

In [137]:
# flatten the dataframe
features_data.columns = [' '.join(col) for col in features_data.columns]
features_data = features_data.reset_index()
features_data.head()

,track_id,chroma_cens kurtosis 01,chroma_cens kurtosis 02,chroma_cens kurtosis 03,chroma_cens kurtosis 04,chroma_cens kurtosis 05,chroma_cens kurtosis 06,chroma_cens kurtosis 07,chroma_cens kurtosis 08,chroma_cens kurtosis 09,...,tonnetz std 04,tonnetz std 05,tonnetz std 06,zcr kurtosis 01,zcr max 01,zcr mean 01,zcr median 01,zcr min 01,zcr skew 01,zcr std 01
0,2,7.180653,5.230309,0.249321,1.347620,1.482478,0.531371,1.481593,2.691455,0.866868,...,0.054125,0.012226,0.012111,5.758890,0.459473,0.085629,0.071289,0.000000,2.089872,0.061448
1,3,1.888963,0.760539,0.345297,2.295201,1.654031,0.067592,1.366848,1.054094,0.108103,...,0.063831,0.014212,0.017740,2.824694,0.466309,0.084578,0.063965,0.000000,1.716724,0.069330
2,5,0.527563,-0.077654,-0.279610,0.685883,1.937570,0.880839,-0.923192,-0.927232,0.666617,...,0.040730,0.012691,0.014759,6.808415,0.375000,0.053114,0.041504,0.000000,2.193303,0.044861
3,10,3.702245,-0.291193,2.196742,-0.234449,1.367364,0.998411,1.770694,1.604566,0.521217,...,0.074358,0.017952,0.013921,21.434212,0.452148,0.077515,0.071777,0.000000,3.542325,0.040800
4,20,-0.193837,-0.198527,0.201546,0.258556,0.775204,0.084794,-0.289294,-0.816410,0.043851,...,0.095003,0.022492,0.021355,16.669037,0.469727,0.047225,0.040039,0.000977,3.189831,0.030993


## track data

In [138]:
tracks = utils.load('fma_metadata/tracks.csv')

In [139]:
tracks.columns

MultiIndex([( 'album',          'comments'),
            ( 'album',      'date_created'),
            ( 'album',     'date_released'),
            ( 'album',          'engineer'),
            ( 'album',         'favorites'),
            ( 'album',                'id'),
            ( 'album',       'information'),
            ( 'album',           'listens'),
            ( 'album',          'producer'),
            ( 'album',              'tags'),
            ( 'album',             'title'),
            ( 'album',            'tracks'),
            ( 'album',              'type'),
            ('artist', 'active_year_begin'),
            ('artist',   'active_year_end'),
            ('artist', 'associated_labels'),
            ('artist',               'bio'),
            ('artist',          'comments'),
            ('artist',      'date_created'),
            ('artist',         'favorites'),
            ('artist',                'id'),
            ('artist',          'latitude'),
          

In [140]:
## using genre_top as target label
tracks["track", "genre_top"], tracks["track", "genre_top"].shape

(track_id
 2         Hip-Hop
 3         Hip-Hop
 5         Hip-Hop
 10            Pop
 20            NaN
            ...   
 155316       Rock
 155317       Rock
 155318       Rock
 155319       Rock
 155320        NaN
 Name: (track, genre_top), Length: 106574, dtype: category
 Categories (16, object): ['Blues', 'Classical', 'Country', 'Easy Listening', ..., 'Pop', 'Rock', 'Soul-RnB', 'Spoken'],
 (106574,))

In [141]:
# drop tracks with no genre_top
label = tracks["track", "genre_top"].dropna().reset_index()
label.columns = [''.join(col) for col in label.columns]
label = label.rename(columns={"trackgenre_top" : "genre"})
label.shape, label

((49598, 2),
        track_id    genre
 0             2  Hip-Hop
 1             3  Hip-Hop
 2             5  Hip-Hop
 3            10      Pop
 4           134  Hip-Hop
 ...         ...      ...
 49593    155315     Rock
 49594    155316     Rock
 49595    155317     Rock
 49596    155318     Rock
 49597    155319     Rock
 
 [49598 rows x 2 columns])

In [142]:
features_data = features_data.merge(label, how="inner", on="track_id")
features_data = features_data.drop(columns="track_id")
features_data

,chroma_cens kurtosis 01,chroma_cens kurtosis 02,chroma_cens kurtosis 03,chroma_cens kurtosis 04,chroma_cens kurtosis 05,chroma_cens kurtosis 06,chroma_cens kurtosis 07,chroma_cens kurtosis 08,chroma_cens kurtosis 09,chroma_cens kurtosis 10,...,tonnetz std 05,tonnetz std 06,zcr kurtosis 01,zcr max 01,zcr mean 01,zcr median 01,zcr min 01,zcr skew 01,zcr std 01,genre
0,7.180653,5.230309,0.249321,1.347620,1.482478,0.531371,1.481593,2.691455,0.866868,1.341231,...,0.012226,0.012111,5.758890,0.459473,0.085629,0.071289,0.000000,2.089872,0.061448,Hip-Hop
1,1.888963,0.760539,0.345297,2.295201,1.654031,0.067592,1.366848,1.054094,0.108103,0.619185,...,0.014212,0.017740,2.824694,0.466309,0.084578,0.063965,0.000000,1.716724,0.069330,Hip-Hop
2,0.527563,-0.077654,-0.279610,0.685883,1.937570,0.880839,-0.923192,-0.927232,0.666617,1.038546,...,0.012691,0.014759,6.808415,0.375000,0.053114,0.041504,0.000000,2.193303,0.044861,Hip-Hop
3,3.702245,-0.291193,2.196742,-0.234449,1.367364,0.998411,1.770694,1.604566,0.521217,1.982386,...,0.017952,0.013921,21.434212,0.452148,0.077515,0.071777,0.000000,3.542325,0.040800,Pop
4,0.918445,0.674147,0.577818,1.281117,0.933746,0.078177,1.199204,-0.175223,0.925482,1.438509,...,0.016322,0.015819,4.731087,0.419434,0.064370,0.050781,0.000000,1.806106,0.054623,Hip-Hop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49593,-0.319850,0.653044,1.300360,-0.606061,-1.109302,-1.006683,0.059643,0.472580,-0.856436,-0.524676,...,0.016631,0.014705,6.947788,0.188477,0.037708,0.035156,0.002930,1.764233,0.018679,Rock
49594,-0.490129,0.463834,2.321970,-0.084352,1.662914,2.115189,-0.237794,5.695442,0.830353,1.951819,...,0.022547,0.019816,4.448255,0.172852,0.028773,0.028320,0.003906,0.955388,0.012385,Rock
49595,-0.461559,-0.229601,-0.496632,-0.422033,0.130612,-0.263825,-0.628103,-0.082687,-0.229483,-0.492753,...,0.023548,0.026527,3.270612,0.196289,0.031116,0.027832,0.002441,1.283060,0.019059,Rock
49596,0.552473,-0.110498,-0.532014,0.263131,-0.224011,-0.530972,1.713526,1.418444,1.325197,0.120333,...,0.017540,0.020471,2.356727,0.212891,0.038450,0.037109,0.003418,0.828569,0.017904,Rock


# Random Forest

In [143]:
# split data into training and test
y = features_data["genre"].copy(deep=True)
X = features_data.drop(columns="genre").copy(deep=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((37198, 518), (37198,), (12400, 518), (12400,))

## preliminary result

In [144]:
# classifier fitting
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

RandomForestClassifier()

In [145]:
# classifier testing
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.6223387096774193

## search for best model